In [1]:
import pandas as pd

In [2]:
users = pd.read_csv('../data/users.csv.bak')
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   user_id   68092 non-null  int64  
 1   location  68092 non-null  object 
 2   age       40259 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ MB


In [3]:
users.head()

,user_id,location,age
0,8,"timmins, ontario, canada",NaN
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",NaN
3,67544,"toronto, ontario, canada",30.0
4,85526,"victoria, british columbia, canada",36.0


### user location 나눠주기

In [4]:
location_split = users['location'].str.split(',', expand=True, n=4)
location_split.iloc[[1044]]

,0,1,2,3,4
1044,paris,france,ile de france,france,None


In [5]:
location_split = users['location'].str.split(',', expand=True, n=4)

# 각 행의 데이터를 오른쪽으로 이동시키기 위한 이동 칸 수 계산
shift_amount = 5 - location_split.count(axis=1)

# 각 행을 오른쪽으로 이동
location_split = location_split.apply(lambda x: x.shift(shift_amount[x.name]), axis=1)
location_split.iloc[[1044]]

,0,1,2,3,4
1044,None,paris,france,ile de france,france


In [6]:
new_column_names = ['location_part1', 'location_part2', 'location_part3', 'location_part4', 'location_part5']
for i, col in enumerate(new_column_names):
    users[col] = location_split[i]

users

,user_id,location,age,location_part1,location_part2,location_part3,location_part4,location_part5
0,8,"timmins, ontario, canada",NaN,None,None,timmins,ontario,canada
1,11400,"ottawa, ontario, canada",49.0,None,None,ottawa,ontario,canada
2,11676,"n/a, n/a, n/a",NaN,None,None,n/a,n/a,n/a
3,67544,"toronto, ontario, canada",30.0,None,None,toronto,ontario,canada
4,85526,"victoria, british columbia, canada",36.0,None,None,victoria,british columbia,canada
...,...,...,...,...,...,...,...,...
68087,278376,"danville, pennsylvania, usa",54.0,None,None,danville,pennsylvania,usa
68088,278621,"victoria, delaware, canada",74.0,None,None,victoria,delaware,canada
68089,278636,"irvington, alabama, usa",NaN,None,None,irvington,alabama,usa
68090,278659,"vancouver, washington, usa",33.0,None,None,vancouver,washington,usa


In [7]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         68092 non-null  int64  
 1   location        68092 non-null  object 
 2   age             40259 non-null  float64
 3   location_part1  10 non-null     object 
 4   location_part2  295 non-null    object 
 5   location_part3  68092 non-null  object 
 6   location_part4  68092 non-null  object 
 7   location_part5  68092 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 4.2+ MB


### user가 주는 점수 개수, 평균, 최빈, 중앙, 표준편차차

In [8]:
train_ratings = pd.read_csv('../data/train_ratings.csv.bak')
train_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306795 entries, 0 to 306794
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  306795 non-null  int64 
 1   isbn     306795 non-null  object
 2   rating   306795 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 7.0+ MB


In [9]:
user_rating_counts = train_ratings.groupby('user_id')['rating'].count()
users = users.merge(user_rating_counts, on='user_id', how='left')
users = users.rename(columns={'rating': 'user_rating_count'})
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            68092 non-null  int64  
 1   location           68092 non-null  object 
 2   age                40259 non-null  float64
 3   location_part1     10 non-null     object 
 4   location_part2     295 non-null    object 
 5   location_part3     68092 non-null  object 
 6   location_part4     68092 non-null  object 
 7   location_part5     68092 non-null  object 
 8   user_rating_count  59803 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 4.7+ MB


In [10]:
# user_rating_means = train_ratings.groupby('user_id')['rating'].mean()
# users = users.merge(user_rating_means, on='user_id', how='left')
# users = users.rename(columns={'rating': 'user_rating_mean'})
# users.info()

In [11]:
user_rating_modes = train_ratings.groupby('user_id')['rating'].agg(lambda x: x.mode().iloc[0])
users = users.merge(user_rating_modes, on='user_id', how='left')
users = users.rename(columns={'rating': 'user_rating_mode'})
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            68092 non-null  int64  
 1   location           68092 non-null  object 
 2   age                40259 non-null  float64
 3   location_part1     10 non-null     object 
 4   location_part2     295 non-null    object 
 5   location_part3     68092 non-null  object 
 6   location_part4     68092 non-null  object 
 7   location_part5     68092 non-null  object 
 8   user_rating_count  59803 non-null  float64
 9   user_rating_mode   59803 non-null  float64
dtypes: float64(3), int64(1), object(6)
memory usage: 5.2+ MB


In [12]:
user_rating_median = train_ratings.groupby('user_id')['rating'].median()
users = users.merge(user_rating_median, on='user_id', how='left')
users = users.rename(columns={'rating': 'user_rating_median'})
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             68092 non-null  int64  
 1   location            68092 non-null  object 
 2   age                 40259 non-null  float64
 3   location_part1      10 non-null     object 
 4   location_part2      295 non-null    object 
 5   location_part3      68092 non-null  object 
 6   location_part4      68092 non-null  object 
 7   location_part5      68092 non-null  object 
 8   user_rating_count   59803 non-null  float64
 9   user_rating_mode    59803 non-null  float64
 10  user_rating_median  59803 non-null  float64
dtypes: float64(4), int64(1), object(6)
memory usage: 5.7+ MB


In [13]:
# user_rating_stds = train_ratings.groupby('user_id')['rating'].std()
# users = users.merge(user_rating_stds, on='user_id', how='left')
# users = users.rename(columns={'rating': 'user_rating_std'})
# users.info()

In [14]:
users.to_csv('../data/users_m1.csv', index=False)
!cp ../data/users_m1.csv ../data/users.csv